## CS431/631 Big Data Infrastructure
### Winter 2020 - Assignment 1
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** _Huaqing Fang_
* **ID:** _20749690_

---
#### Overview
For this assignment, you will be using Python to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) that you used for Assignment 0.   You will also need the Python tokenizer module, `simple_tokenize.py`.

If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

For this assignment, the "events" that we are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.

Your main task for this assignment is to write Python code to analyze the PMI of tokens from Shakespeare's plays.    Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



---
#### Question 1  (2/10 marks):

Before writing code to handle the PMI queries, start writing some code to answer some simpler questions that give an
idea of how big the PMI analysis problem will be.   The box below contains some starter code that reads in the 'Shakespeare.txt' file and tokenizes it one line at time.   (This is the same code you started with for Assignment 0.)  Extend this code to determine (a) the number of *distinct* tokens that exist in 'Shakespeare.txt', and (b) the number of 
distinct token pairs that exist in 'Shakespeare.txt'.  For the purposes of this question, consider the token pair $x,y$ to be distinct from the pair $y,x$, i.e., count them both.   Ignore token pairs of the form $x,x$.

In [1]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize

# Now, let's tokenize Shakespeare's plays
filename = 'Shakespeare.txt'
dic = {} # to store all the distinct words in the text and their respective frequencies
count = 0
dic_pairs = {} # to store all the distinct pairs in the text and their respective frequencies
pairs_count = 0
line_count = 0

with open(filename) as f:
    for line in f:
        # tokenize, one line at a time
        t = simple_tokenize(line)
        #print(t)
        line_word = [] # to store the distinct words in the line t
        line_count +=1
        for i in t:
            #print(i)
            # update dict
            if i in dic:
                dic[i] = dic[i] +1
            else:
                dic[i] = 1;
            count += 1
            if i not in line_word:
                line_word.append(i)
        #count += len(line_word)
        #print(line_word)
        length = len(line_word)
        for i in range(length):
            for j in range(i+1, length):
                tmp1 = (line_word[i], line_word[j])
                tmp2 = (line_word[j], line_word[i])
                if tmp1 in dic_pairs:
                    dic_pairs[tmp1] +=1
                else:
                    dic_pairs[tmp1] = 1
                    
                if tmp2 in dic_pairs:
                    dic_pairs[tmp2] +=1
                else:
                    dic_pairs[tmp2] = 1
                pairs_count +=2
                     
print("the total number of distinct words is {0}".format(len(dic)))
print("the total number of distinct pairs is {0}".format(len(dic_pairs)))
#print(line_count)
#print(count)
            
            
            
        

# extend this code to answer Question 1.
# when your code is executed, it should print the number of distinct tokens and the number of distinct token pairs

the total number of distinct words is 25975
the total number of distinct pairs is 1969760


---

#### Question 2 (6/10 marks):
Next, write Python code to answer the one-token and two-token queries described above, for 'Shakespeare.txt'.   The code cell below contains some starter code that implements a simple text-based query interface.  It allows a user to ask a series of one-token or two-token queries.   Try running the starter code to get a sense of how the interface behaves.    

Your task is to write code to read and tokenize 'Shakespeare.txt', record information that will allow both types of PMI queries to be answered, and then answer queries that are posed through the query interface.  Make sure that your code is well commented, so that it will be clear to the markers.

If you cannot get answers to both types of queries working, try to get at least one type working, for partial credit.


In [ ]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10
from math import log

###################################################################################################################
#  replace this with your PMI analysis code, so that you can support the user interface below
#  it should read and tokenize Shakespeare.txt, and store enough information in Python data structures
#  to allow you to answer the PMI queries below
###################################################################################################################

###################################################################################################################
#  the user interface below defines the types of PMI queries that users can ask
#  you will need to modify it - where indicated - to access the results of your PMI analysis (above)
#  so that the queries can be answered
###################################################################################################################



# please run cell one before running this cell, some of the data are computed in the previous cell
# all pairs and their repective frequencies are stored in dic_pairs in part1
# all words and their repective frequencies are stored in dic computed in part1
                
num_words = len(dic)
num_pairs = len(dic_pairs)
#print(num_words)
#print(num_pairs)

while True:
    q = input("Input 1 or 2 space-separated tokens (return to quit): ")
    if len(q) == 0:
        break
    q_tokens = simple_tokenize(q)
    
    word_l = []
    for i in dic.keys():
        word_l.append(i) # a list of all distinct words
                         # later to be used to find pairs start with a certain word
   
    #print(word_l)
    if len(q_tokens) == 1:
        threshold = 0
        while threshold <= 0:
            try:
                threshold = int(input("Input a positive integer frequency threshold: "))
            except ValueError:
                print("Threshold must be a positive integer!")
                continue
                
        word = q_tokens[0]
        if word not in dic:
            print("word does not exist")
            continue
            
        tmp_dic  = {}  # dictionary in which the first word of the pairs are the same
        for i in word_l:  # for loop to construct tmp_dic
            pair = (word,i)
            if pair not in dic_pairs:
                continue
                
           # then the pair exists
            px = dic[i] / count
            py = dic[word] / count
            pxy = dic_pairs[pair] / line_count
            pmi = log( pxy / (px * py), 10)
            #print(pmi)
            tmp_dic[pair] = pmi
            
        #sort by values
        sorted_dic = sorted(tmp_dic.items(), key=lambda kv: kv[1])
        #print(sorted_dic)
        
        num = dic[word]
        print("  n({0}) = {1}".format(word, num))
        print("  high PMI tokens with respect to {0} (threshold: {1}):".format(q_tokens[0],threshold))
        
        founded = 0
        for i in range(1, len(sorted_dic)):
            if founded == 5:
                break
            token = sorted_dic[-i][0][1]
            val = sorted_dic[-i][1]
            # deal with the thredhold
            if dic_pairs[(word,token)] < threshold:
                continue
                
            print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".format(q_tokens[0], token, dic_pairs[(word,token)], val)) 
            # founded 1
            founded +=1
            if founded < 5 and i == len(sorted_dic) -1:
                print("no more satisficed pairs")
                break
            
        # Put code here to answer a One-Token Query with token q_tokens[0] and the specified threshold,
        # and output the result.
        # The print() statements below exist to show you the desired output format.
        # Replace them with your own output code, which should produce results in a similar format.
       ## print("  n({0}) = XXX".format(q_tokens[0]))
       # print("  high PMI tokens with respect to {0} (threshold: {1}):".format(q_tokens[0],threshold))
       # print("    n({0},token1) = XXX,  PMI({0},token1) = Y.YYY".format(q_tokens[0]))    
       # print("    n({0},token2) = XXX,  PMI({0},token2) = Y.YYY".format(q_tokens[0]))
       # print("    n({0},token3) = XXX,  PMI({0},token3) = Y.YYY".format(q_tokens[0]))
       # print("    n({0},token4) = XXX,  PMI({0},token4) = Y.YYY".format(q_tokens[0]))
       # print("    n({0},token5) = XXX,  PMI({0},token5) = Y.YYY".format(q_tokens[0]))
        # in the above, all XXX values should be at least as large as the threshold

    elif len(q_tokens) == 2:
        # Put code here to answer a Two-Token Query with tokens q_tokens[0] and q_tokens[1]
        # As was the case for the One-Token query, the print statements below show the desired output format
        # Replace them with your own output code
        tmp = (q_tokens[0], q_tokens[1])
        px = dic[q_tokens[0]] / count
        py = dic[q_tokens[1]] / count
        pxy = dic_pairs[tmp] / line_count
        pmi = log( pxy / (px * py), 10)
        print("  n({0},{1}) = {2}".format(q_tokens[0],q_tokens[1],dic_pairs[tmp]))
        print("  PMI({0},{1}) = {2}".format(q_tokens[0],q_tokens[1],pmi))
    else:
        print("Input must consist of 1 or 2 space-separated tokens!")


Input 1 or 2 space-separated tokens (return to quit): love


---

#### Question 3 (2/10 marks):

Suppose that you try to run your PMI analysis on larger files:  say, 10 times, or 100 times, or 1000 times larger than 'Shakespeare.txt'.    As the input file grows larger, what will happen to the execution of your program?   Will it get slower?   How much slower?   Will it eventually fail to run?   If so, why?

In the cell below, briefly (one or two paragraphs), discuss what will happen if the input to your analysis grows.  We're not looking for an exact or empirical analysis of the behaviour of your program as a function of input size.  Rather, we are looking for a discussion of trends and limits.

#### Answer to Question 3:

*My program will get much slower.
The most costly part of my program is to constructing dictionaries. Other than that, searching and checking for existance are all constant time. Contrsuting a dictionary with all distinct words and their repective frequencies is O(n) where n is the length the text. Construct a dictionary with all distinct pairs and their repective frequencies is quadratic with respect to the number of word in each line. Essentially, it is O(n^2) where n is the length of text.
Therefore, my program runs quadratically slower as the size of text increase. Eventually, when my machine runs out of memory, the program fails*


---
Don't forget to save your workbook!   (It's a good idea to do this regularly, while you are working.)   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.